Imports added for web scraping

In [ ]:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd

Web page containing Scorecard links read

In [ ]:
match_schedule_html = open('match_schedule_html.txt', 'r')
soup = BeautifulSoup(match_schedule_html, 'lxml')
match_schedule_html.close()

Scorecard links extracted

In [ ]:
links = soup.select('a')
scorecards = [scorecard['href'] for scorecard in links]
scorecards_links = []
for link in scorecards:
  if ('Scorecard.aspx' in link):
    scorecards_links.append(link)
scorecards_links.pop(0)
scorecards_links[80]

Home Team Score

In [ ]:
url = 'https://orsca.ca/Pages/Scorecard.aspx?MatchID=1802'
reqs = requests.get(url, verify=False) # setting verify to False by passes the SSL Certificate check
scorecard_soup = BeautifulSoup(reqs.text, 'lxml')

home_table = scorecard_soup.find('table', {'id':'Home'})
home_df = pd.DataFrame(columns=['Home Inning', 'Runs', 'How Out', 'O', 'R', 'W', 'H'])
home_table_row = ''

for row in home_table.find_all('tr'):
  home_table_row = row
  home_scorecard_row_soup = BeautifulSoup(str(home_table_row), 'lxml')
  home_scorecard_td = home_scorecard_row_soup.find('td')

  # extract values to insert into new row of home team score dataframe
  home_inning = home_scorecard_td.find_all('div')[1].text
  home_runs = home_scorecard_td.find_all('div')[2].text
  home_how_out = home_scorecard_td.find_all('div')[3].text
  home_o = home_scorecard_td.find_all('div')[4].text
  home_r = home_scorecard_td.find_all('div')[5].text
  home_w = home_scorecard_td.find_all('div')[6].text
  home_h = home_scorecard_td.find_all('div')[7].text

  home_list_row = [home_inning, home_runs, home_how_out, home_o, home_r, home_w, home_h]
  home_df.loc[len(home_df)] = home_list_row

home_df.drop(0, axis=0, inplace=True)
# home_df = home_df.style.hide_index()
home_df

Away Team Score

In [ ]:
visitor_table = scorecard_soup.find('table', {'id':'Visitor'})
visitor_df = pd.DataFrame(columns=['Visitor Inning', 'Runs', 'How Out', 'O', 'R', 'W', 'H'])
visitor_table_row = ''

for row in visitor_table.find_all('tr'):
  visitor_table_row = row
  visitor_scorecard_row_soup = BeautifulSoup(str(visitor_table_row), 'lxml')
  visitor_scorecard_td = visitor_scorecard_row_soup.find('td')

  # extract values to insert into new row of visitor team score dataframe
  visitor_inning = visitor_scorecard_td.find_all('div')[1].text
  visitor_runs = visitor_scorecard_td.find_all('div')[2].text
  visitor_how_out = visitor_scorecard_td.find_all('div')[3].text
  visitor_o = visitor_scorecard_td.find_all('div')[4].text
  visitor_r = visitor_scorecard_td.find_all('div')[5].text
  visitor_w = visitor_scorecard_td.find_all('div')[6].text
  visitor_h = visitor_scorecard_td.find_all('div')[7].text

  visitor_list_row = [visitor_inning, visitor_runs, visitor_how_out, visitor_o, visitor_r, visitor_w, visitor_h]
  visitor_df.loc[len(visitor_df)] = visitor_list_row

visitor_df.drop(0, axis=0, inplace=True)
# visitor_df = visitor_df.style.hide_index()
visitor_df

Match Overview

In [ ]:
overview_table = scorecard_soup.find('table', {'id':'hd1'})
team1 = overview_table.find_all('div', {'class':'team'})[0].text
team2 = overview_table.find_all('div', {'class':'team'})[1].text
team1_points = overview_table.find_all('div', {'class':'team'})[2].text
team2_points = overview_table.find_all('div', {'class':'team'})[3].text
date = overview_table.find_all('div', {'class':'Comments'})[0].text
venue = overview_table.find_all('div', {'class':'Comments'})[1].text
comments = overview_table.find_all('div', {'class':'Comments'})[2].text

overview_df = pd.DataFrame({'Teams':[team1, 'vs.', team2], 
                            'Points':[team1_points, '', team2_points], 
                            'Date':[date, '', ''], 'Venue':[venue, '', ''], 
                            'Comments':[comments, '', '']}).transpose()
overview_df.columns = ['', '', '']
overview_df


Add the Dataframes to the Excel Workbook

In [ ]:
with pd.ExcelWriter("2022_t15_scorecards.xlsx", mode="a", engine="openpyxl", if_sheet_exists="replace") as writer:
  overview_df.to_excel(writer, sheet_name="Aug_14_ParisSun_Pegasus", startrow=0, startcol=0)
  home_df.to_excel(writer, sheet_name="Aug_14_ParisSun_Pegasus", startrow=7, startcol=0, index=False)
  visitor_df.to_excel(writer, sheet_name="Aug_14_ParisSun_Pegasus",startrow=23, startcol=0, index=False)